In [1]:
%run header.ipynb

/usr/local/lib/python3.7/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [2]:
train_data = parquet.read_table(input_path + '/textsTrain').to_pandas()
train_data.rename(columns = {'instanceId_objectId':'objectId'}, inplace = True)

/usr/local/lib/python3.7/site-packages/pyarrow/pandas_compat.py:752: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels


In [3]:
test_data = parquet.read_table(input_path + '/textsTest').to_pandas()
test_data.rename(columns = {'instanceId_objectId':'objectId'}, inplace = True)

In [5]:
train_data.tail(200)

,instanceId_userId,instanceId_objectType,objectId,feedback,audit_clientType,audit_timestamp,metadata_ownerId,metadata_createdAt,date
19037404,15660337,Video,858090,[Ignored],API,1521627455881,84890,1521374351000,2018-03-21
19037405,15660484,Post,20443008,[Ignored],MOB,1521661345941,6653,1521627070000,2018-03-21
19037406,15660670,Post,37236405,[Ignored],API,1521643884184,48570,1521487327000,2018-03-21
19037407,15660670,Video,9313861,[Ignored],API,1521647172656,23430,1521639104000,2018-03-21
19037408,15661339,Post,25638393,[Ignored],WEB,1521609668802,72578,1520964255000,2018-03-21
19037409,15661615,Post,14797333,[Ignored],API,1521619647918,25670,1521602817000,2018-03-21
19037410,15661981,Video,507070,[Disliked],API,1521599920055,64054,1521456635000,2018-03-21
19037411,15662344,Post,21418250,[Ignored],MOB,1521609795018,851,1521335726000,2018-03-21
19037412,15663610,Post,34711941,[Ignored],API,1521634307998,9182,1521605125000,2018-03-21
19037413,15663631,Post,10981255,[Ignored],API,1521630749558,18998,1521528312000,2018-03-21


In [7]:
train_data.instanceId_objectType.unique()

array(['Post', 'Video', 'Photo'], dtype=object)

In [8]:
train_data.audit_clientType.unique()

array(['WEB', 'API', 'MOB'], dtype=object)

In [9]:
def proc_data_inplace(data):
    data['instanceId_objectType'] = np.array(data.instanceId_objectType.replace({
        'Post': 0,
        'Video': 1,
        'Photo': 2,
    }), np.int8)
    data['audit_clientType'] = np.array(data.audit_clientType.replace({
        'WEB': 0,
        'API': 1,
        'MOB': 2,
    }), np.int8)
    if 'feedback' in data.columns:
        for action in ('Liked', 'Clicked', 'Viewed', 'Disliked', 'ReShared', 'Ignored'):
            train_data[action.lower()] = np.array(train_data['feedback'].apply(lambda x: 1 if action in x else 0).values, np.int8)
        data.drop('feedback', 1, inplace=True)

In [10]:
proc_data_inplace(train_data)

In [11]:
proc_data_inplace(test_data)

In [12]:
train_data.head()

,instanceId_userId,instanceId_objectType,objectId,audit_clientType,audit_timestamp,metadata_ownerId,metadata_createdAt,date,liked,clicked,viewed,disliked,reshared,ignored
0,1618,0,25814780,0,1517458217938,81088,1517454825000,2018-02-01,0,1,0,0,0,1
1,2122,0,10027037,1,1517488844356,4016,1517423778000,2018-02-01,0,0,0,0,0,1
2,3400,0,21687059,0,1517510326582,33546,1517509621000,2018-02-01,0,0,0,0,0,1
3,4246,0,9938784,1,1517464554986,74384,1517218637000,2018-02-01,0,0,0,0,0,1
4,4516,0,22275114,0,1517467871914,32254,1517247354000,2018-02-01,0,0,0,0,0,1


In [13]:
train_data.dtypes

instanceId_userId           int32
instanceId_objectType        int8
objectId                    int32
audit_clientType             int8
audit_timestamp             int64
metadata_ownerId            int32
metadata_createdAt          int64
date                     category
liked                        int8
clicked                      int8
viewed                       int8
disliked                     int8
reshared                     int8
ignored                      int8
dtype: object

In [14]:
train_data.to_pickle(output_path + '/train_data')
test_data.to_pickle(output_path + '/test_data')

In [4]:
train_data.head(100).feedback

0               [Clicked, Ignored]
1                        [Ignored]
2                        [Ignored]
3                        [Ignored]
4                        [Ignored]
5                        [Ignored]
6                          [Liked]
7                        [Ignored]
8                        [Ignored]
9                        [Ignored]
10                       [Clicked]
11                       [Ignored]
12                       [Ignored]
13                         [Liked]
14    [Clicked, ReShared, Ignored]
15                       [Ignored]
16                       [Ignored]
17                       [Ignored]
18                       [Clicked]
19                       [Clicked]
20             [Disliked, Ignored]
21             [Disliked, Ignored]
22                       [Clicked]
23                       [Clicked]
24                       [Clicked]
25                       [Clicked]
26                       [Ignored]
27                         [Liked]
28              [Cli